In [ ]:
import torch
from vggt.models.vggt import VGGT
from vggt.utils.load_fn import load_and_preprocess_images

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16

model = VGGT()
model.load_state_dict(torch.load("weights/model.pt"))
model.to(device)

In [ ]:
import os
image_folder='examples/kitchen/images'
image_names=[]
for image in os.listdir(image_folder):
    image_names.append(os.path.join(image_folder, image))

In [ ]:
# Load and preprocess example images (replace with your own image paths)
# image_names = ["path/to/imageA.png", "path/to/imageB.png", "path/to/imageC.png"]  
images = load_and_preprocess_images(image_names).to(device)

with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=dtype):
        # Predict attributes including cameras, depth maps, and point maps.
        predictions = model(images)

In [ ]:
print(predictions.keys())
print(predictions['depth'].shape)
print(predictions['depth_conf'].shape)